# Part 2: Automatic Ontology Generation

In [1]:
import spacy
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.chunk import ne_chunk
from sklearn import svm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.cluster import KMeans
from collections import defaultdict

### Preprocessing

In [2]:
def preprocess_text(text):
    # Tokenization into sentences and words
    sentences = sent_tokenize(text)
    words = [word_tokenize(sentence) for sentence in sentences]
    
    # Removing stopwords
    stop_words = set(stopwords.words("english"))
    filtered_words = [[word for word in sentence if word.lower() not in stop_words] for sentence in words]
    
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    lemmatized_words = [[lemmatizer.lemmatize(word) for word in sentence] for sentence in filtered_words]
    
    return lemmatized_words

### Extracting concepts using Named Entity Recognition (NER)

In [3]:
def extract_entities(text):
    tagged_words = nltk.pos_tag(text)
    named_entities = ne_chunk(tagged_words, binary=True)  
    
    entities = []
    for entity in named_entities:
        if hasattr(entity, 'label') and entity.label:
            entity_text = ' '.join(word for word, tag in entity.leaves())
            entities.append(entity_text)
    
    return entities

### Extracting relationships using dependency parsing

In [4]:
def extract_relationships(text):
    nlp = spacy.load('en_core_web_sm')
    nlp.max_length = 6000000  
    doc = nlp(text)
    relationships = []
    for token in doc:
        if token.dep_ == 'amod' and token.head.pos_ == 'NOUN':
            relationships.append((token.text, token.head.text))
    
    return relationships

### Removing duplicates

In [5]:
def remove_duplicates(concepts):
    unique_concepts = []
    for concept in concepts:
        words = concept.split()
        if not any(set(words) == set(c.split()) for c in unique_concepts):
            unique_concepts.append(concept)
    return unique_concepts

### Concepts and relationships

In [6]:
file_path = "small.txt"
encoding="utf-8"
    
with open(file_path, "r", encoding=encoding) as file:
      text = file.read()

preprocessed_text = preprocess_text(text)
preprocessed_text_flat = [word for sentence in preprocessed_text for word in sentence]
concepts = extract_entities(preprocessed_text_flat)
#relationships = extract_relationships(' '.join(preprocessed_text_flat))

# Removing duplicates
unique_concepts = list(set(concepts))

print("Concepts:", unique_concepts)

Concepts: ['Baseline Glycosylated Hemoglobin', 'Minute Walk', 'HbA1c', 'Adjusted Rates', 'CLr', 'PMT', 'Diabetes Knowledge Baseline', 'PEFR', 'Hypoglycaemic', 'HFSws', 'Total', 'Hemoglobin', 'Baseline Fasting Blood Glucose', 'QoL', 'Hyperglycaemia Avoidance', 'Incidences Adverse', 'DCCT', 'Severe Hypoglycemia', 'Readiness Change Baseline', 'Blood Glucose', 'HbA1c Less', 'Weight Loss Baseline', 'Dose', 'Blood', 'Weight Loss', 'Glycemic Control Baseline', 'A1c', 'Chinese', 'Treg', 'AUClast', 'GIP', 'Glycosylated Hemoglobin', 'Alzheimer', 'AEs', 'Lipid', 'Macro', 'Gastric Bypass', 'Hospital Anxiety Depression Scale', 'TCR', 'Haemoglobin', 'CTSIB', 'Baseline', 'Quality', 'Serious', 'Glucose', 'Past Medical History', 'Diabetes', 'FEV1', 'SMBG', 'Baseline Mean Weight', 'FPG', 'DTSQ', 'Insulin', 'ASP1941', 'ADDQoL', 'AUCinf', 'Adverse', 'Clinical Test Sensory Interaction Balance', 'Baseline Blood Glucose Measured Meal Tolerance Test', 'SCOUT DS', 'AUC', 'HADS', 'BMI', 'FVC']


### Concept categorization using Support Vector Machines (SVM)

In [7]:
# Loading the pre-trained spaCy model
nlp = spacy.load("en_core_web_sm")

# Training data (labeled concepts with their categories)
training_data = [
    ("Blood glucose", "Biomarker"),("Leptin", "Biomarker"),("HbA1c", "Biomarker"),("Insulin", "Biomarker"),("Free Fatty Acids", "Biomarker"),
    ("High-Sensivity R-Reactive Protein", "Biomarker"),("Homa Beta-cell Function (HOMA-B)", "Biomarker"),("Interleukin-18", "Biomarker"),
    ("Lipoproteins", "Biomarker"),("Plasma", "Biomarker"),("Plasminogen Activator Inhibitor-1", "Biomarker"),("Epinephrine", "Biomarker"),
    ("highsensitivity C-reactive protein", "Biomarker"),("hsCRP", "Biomarker"),("Adiponectin", "Biomarker"),("ApoB", "Biomarker"),("Waist Circumference", "Endpoint score"),
    ("Apolipoprotein B", "Biomarker"),("Blood Pressure", "Biomarker"),("Blood lipids", "Biomarker"),("C-peptide", "Biomarker"),("Adverse events", "Complications"),
    ("Cholesterol", "Biomarker"),("BMI", "Endpoint score"),("AUC", "Endpoint score"),("Rate", "Endpoint score"),("index", "Endpoint score"),("indices", "Endpoint score"),("Level", "Endpoint score"),("concentration", "Endpoint score"),
    ("Alloreactive Antibody Levels", "Endpoint score"),("Ankle Brachial Index", "Endpoint score"),("DASS-C21scale score", "Endpoint score"),("glucose levels", "Endpoint score"),
    ("Expiration/Inspiration Ratio", "Endpoint score"),("Fluid cell count", "Endpoint score"),("density", "Endpoint score"),("ratio", "Endpoint score"),("Variability", "Endpoint score"),
    ("Hyperglycemic Events Frequency", "Endpoint score"), ("Urinary Output", "Endpoint score"),("Diabetic retinopathy", "Complications"),("VO2max", "Endpoint score"),
    ("Cardiovascular disease", "Complications"),("Fatigue", "Complications"),("Depression", "Complications"),("Meal Tolerance Test", "Measurement tool"),
    ("Diabetic ketoacidosis", "Complications"),("Glucometer", "Measurement tool"),("Electromyography", "Measurement tool"),("metabolomics", "Measurement tool"),("biomedicalscreen", "Measurement tool"),
    ("1H-MRS", "Measurement tool"),("Antibody titer", "Measurement tool"),("CGM", "Measurement tool"),("Diabetes Numeracy Test", "Measurement tool"),("test", "Measurement tool"),("analysis", "Measurement tool"),
    ("Hyperinsulinemic-euglycemic clamp", "Measurement tool"),("Pharmacodynamics", "Measurement tool"),("Stadiometer", "Measurement tool"),("C-peptide levels", "Endpoint score"),
    ("Seca 203 Circumference Measuring Tape", "Measurement tool"),("Self-monitoring of blood glucose", "Measurement tool"),("MoCA", "Questionnaire"),("PHQ-9", "Questionnaire"),
    ("Quality of Life", "Questionnaire"),("DES-DSF", "Questionnaire"),("Diabetes distress survey", "Questionnaire"),("Questionnaire", "Questionnaire"),("survey", "Questionnaire"),
    ("Mini Nutritional Assessment", "Questionnaire"),("DTSQ", "Questionnaire"),("Confidence in Diabetes Self-Management", "Questionnaire"),("GAD-7", "Questionnaire"),
    ("Diabetes knowledge questionnaire", "Questionnaire"),("EuroQol 5D5L", "Questionnaire"),("General Anxiety Disorder-7", "Questionnaire"),("Knowledge", "Questionnaire"),
    ("Fear Survey II", "Questionnaire"),("Post Clinical Encounter", "Questionnaire"),("Device satisfaction questionaire", "Questionnaire"),("WHO-5", "Questionnaire"),
    ("Food Frequency Questionnaire", "Questionnaire"), ("Score", "Endpoint score"),("Twenty-four Hour Recall Interview", "Questionnaire"),("Questionnaires", "Questionnaire"),

]

# Extracting the concepts and their categories from the training data
concepts, categories = zip(*training_data)

# Adding an "Other" category
concepts += ("Other",)
categories += ("Other",)

# Creating a pipeline for concept categorization
pipeline = Pipeline([
    ("tfidf", TfidfVectorizer()),
    ("svm", svm.SVC(kernel="linear"))
])

# Fitting the pipeline to the training data
pipeline.fit(concepts, categories)

# Categorizing the concepts using the trained SVM model
predicted_categories = pipeline.predict(unique_concepts)
    
# Grouping concepts by categories
concepts_by_category = defaultdict(list)
for concept, category in zip(unique_concepts, predicted_categories):
    concepts_by_category[category].append(concept)

# Printing the categorized concepts by category
for category, concepts in concepts_by_category.items():
    concept_str = ", ".join(concepts)
    print(f"{category}: {concept_str}")

Biomarker: Baseline Glycosylated Hemoglobin, Minute Walk, HbA1c, Adjusted Rates, CLr, PMT, PEFR, Hypoglycaemic, HFSws, Total, Hemoglobin, Baseline Fasting Blood Glucose, QoL, Hyperglycaemia Avoidance, DCCT, Severe Hypoglycemia, Readiness Change Baseline, Blood Glucose, HbA1c Less, Weight Loss Baseline, Dose, Blood, Weight Loss, Glycemic Control Baseline, A1c, Chinese, Treg, AUClast, GIP, Glycosylated Hemoglobin, Alzheimer, AEs, Lipid, Macro, Gastric Bypass, TCR, Haemoglobin, CTSIB, Baseline, Serious, Glucose, Past Medical History, FEV1, SMBG, Baseline Mean Weight, FPG, Insulin, ASP1941, ADDQoL, AUCinf, SCOUT DS, HADS, FVC
Questionnaire: Diabetes Knowledge Baseline, Hospital Anxiety Depression Scale, Quality, Diabetes, DTSQ
Complications: Incidences Adverse, Adverse
Measurement tool: Clinical Test Sensory Interaction Balance, Baseline Blood Glucose Measured Meal Tolerance Test
Endpoint score: AUC, BMI


### Concept categorization using K-means clustering

In [8]:
# Loading the pre-trained spaCy model
nlp = spacy.load("en_core_web_sm")

# Converting the concepts into numerical features
vectorizer = TfidfVectorizer()
concept_features = vectorizer.fit_transform(unique_concepts)

# Performing K-means clustering
num_clusters = 5
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
kmeans.fit(concept_features)
  
# Creating a dictionary to store concepts by cluster
clustered_concepts = defaultdict(list)

# Assigning concepts to their respective clusters
for concept, label in zip(unique_concepts, kmeans.labels_):
    clustered_concepts[label].append(concept)

# Printing the concepts by cluster
for cluster, concepts in clustered_concepts.items():
    print(f"Cluster {cluster}: {', '.join(concepts)}")

Cluster 1: Baseline Glycosylated Hemoglobin, Hemoglobin, Glycosylated Hemoglobin
Cluster 2: Minute Walk, HbA1c, Adjusted Rates, CLr, PMT, PEFR, Hypoglycaemic, HFSws, Total, QoL, Hyperglycaemia Avoidance, Incidences Adverse, DCCT, Severe Hypoglycemia, HbA1c Less, Dose, Weight Loss, A1c, Chinese, Treg, GIP, Alzheimer, AEs, Lipid, Macro, Gastric Bypass, Hospital Anxiety Depression Scale, TCR, Haemoglobin, CTSIB, Quality, Serious, Past Medical History, Diabetes, FEV1, SMBG, FPG, DTSQ, Insulin, ASP1941, ADDQoL, AUCinf, Adverse, Clinical Test Sensory Interaction Balance, SCOUT DS, AUC, HADS, BMI, FVC
Cluster 3: Diabetes Knowledge Baseline, Readiness Change Baseline, Weight Loss Baseline, Glycemic Control Baseline, Baseline, Baseline Mean Weight
Cluster 4: Baseline Fasting Blood Glucose, Blood Glucose, Blood, Glucose, Baseline Blood Glucose Measured Meal Tolerance Test
Cluster 0: AUClast


C:\Users\marin\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
